In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../Data/Downtown_LA_Weather_Data.csv")

/Users/eoin/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8,10,11,27,28,29,33,34,64,84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.keys()
hourlyCols = ['DATE', 'REPORTTPYE', 'HOURLYVISIBILITY', 'HOURLYDRYBULBTEMPF', 'HOURLYRelativeHumidity','HOURLYPrecip']
dailyCols = ['DATE', 'DAILYMaximumDryBulbTemp', 'DAILYMinimumDryBulbTemp', 'DAILYAverageDryBulbTemp',
              'DAILYAverageRelativeHumidity', 'DAILYSunrise', 'DAILYSunset', 'DAILYPrecip', 'DAILYSnowfall', 
              'DAILYSnowDepth', 'DAILYAverageWindSpeed']

In [44]:
# df.to_csv(path_or_buf="ReducedWeather.csv", columns=colsToKeep)

In [4]:
dailySummaries = df.loc[df["REPORTTPYE"] == "SOD"][dailyCols].copy()
hourlySummaries = df.loc[((df["REPORTTPYE"] == "FM-15") | (df["REPORTTPYE"] == "FM-16"))][hourlyCols].copy()

In [5]:
def heatIndex(T, R):
    c1 = -42.379
    c2 = 2.04901523
    c3 = 10.14333127
    c4 = -0.22475541
    c5 = -6.83783e-3
    c6 = -5.481717e-2
    c7 = 1.22874e-3
    c8 = 8.5282e-4
    c9 = -1.99e-6
    return c1 + c2 * T+c3 * R + c4 *T * R + c5 * T**2 + c6 * R**2 + c7 * T**2 * R + c8 * T * R**2 + c9 * T**2 * R**2


In [68]:
wrapper = lambda row: heatIndex(row["DAILYAverageDryBulbTemp"], row["DAILYAverageRelativeHumidity"])

dailySummaries["DAILYAverageDryBulbTemp"] = pd.to_numeric(dailySummaries["DAILYAverageDryBulbTemp"], errors="coerce")
dailySummaries["DAILYAverageRelativeHumidity"] = pd.to_numeric(dailySummaries["DAILYAverageRelativeHumidity"], errors="coerce")
dailySummaries["DAILYAverageHeatIndex"] = dailySummaries.apply(wrapper, axis=1)

dailySummaries.head()

,DATE,DAILYMaximumDryBulbTemp,DAILYMinimumDryBulbTemp,DAILYAverageDryBulbTemp,DAILYAverageRelativeHumidity,DAILYSunrise,DAILYSunset,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth,DAILYAverageWindSpeed,DAILYAverageHeatIndex
24,2010-01-01 23:59,70.0,51,60.0,44.0,659,1656,0.00,NaN,NaN,0.9,82.598461
49,2010-01-02 23:59,78.0,50,64.0,35.0,659,1657,0.00,NaN,NaN,0.9,78.191896
74,2010-01-03 23:59,79.0,54,66.0,24.0,659,1657,0.00,NaN,NaN,1.3,74.808287
99,2010-01-04 23:59,77.0,56,66.0,24.0,659,1658,0.00,NaN,NaN,2.3,74.808287
124,2010-01-05 23:59,78.0,54,66.0,34.0,659,1659,0.00,NaN,NaN,1.6,77.251239


In [7]:
# dailySummaries.to_csv(path_or_buf="DailyWeather.csv")
# hourlySummaries.to_csv(path_or_buf="HourlyWeather.csv")

In [9]:
hourlySummaries.head()

,DATE,REPORTTPYE,HOURLYVISIBILITY,HOURLYDRYBULBTEMPF,HOURLYRelativeHumidity,HOURLYPrecip,HOURLYHeatIndex
0,2010-01-01 00:47,FM-15,10.00,54.0,47.0,0.00,90.857208
1,2010-01-01 01:47,FM-15,10.00,53.0,51.0,0.00,93.280408
2,2010-01-01 02:47,FM-15,10.00,53.0,48.0,0.00,92.748596
3,2010-01-01 03:47,FM-15,10.00,53.0,48.0,0.00,92.748596
4,2010-01-01 04:47,FM-15,10.00,52.0,53.0,0.00,95.417298


Look for weird data types and values. Convert most rows to numeric

In [18]:
set(hourlySummaries["HOURLYVISIBILITY"])

{nan,
 nan,
 2.5,
 3.0,
 4.0,
 '3.00',
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 5.0,
 '5.00',
 2.0,
 1.75,
 1.0,
 nan,
 '0.50',
 '8.00',
 '0.50V',
 nan,
 '2.50V',
 0.5,
 0.25,
 1.25,
 nan,
 nan,
 nan,
 nan,
 nan,
 '0.75V',
 '0.75',
 '1.75V',
 '0.25',
 '1.50V',
 '1.00',
 '6.00',
 '2.00V',
 '7.00',
 '4.00',
 '1.25V',
 '10.00',
 '1.25',
 '9.00',
 '1.00V',
 '0.00',
 '2.50',
 '1.75',
 '2.00',
 '1.50'}

In [77]:
def conv(string):
    if type(string) == type(str()):
        l = len(string)
        if string[l-1] == 'V' or string[l-1] == 's':
            return float(string[:l-1])
        elif string == 'T':
            return 0
        else:
            return float(string)
    return string

In [78]:
conv("9.00V")

9.0

In [79]:
a = hourlySummaries["HOURLYVISIBILITY"].apply(conv)
hourlySummaries["HOURLYVISIBILITY"] = a

In [80]:
set(hourlySummaries["HOURLYPrecip"])

{0.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [81]:
set(hourlySummaries["HOURLYDRYBULBTEMPF"])

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 14.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 38.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 107.0,
 nan,
 nan,
 111.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 34.0,
 35.0,
 36.0,
 37.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 39.0,
 40.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 na

In [82]:
temps = hourlySummaries["HOURLYDRYBULBTEMPF"].apply(conv)
hourlySummaries["HOURLYDRYBULBTEMPF"] = temps

In [83]:
s = hourlySummaries["HOURLYPrecip"].apply(convs)
# set(s)
hourlySummaries["HOURLYPrecip"] = s

In [84]:
set(hourlySummaries["HOURLYDRYBULBTEMPF"])

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 14.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 38.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 107.0,
 nan,
 nan,
 111.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 34.0,
 35.0,
 36.0,
 37.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 39.0,
 40.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 na

In [85]:
set(hourlySummaries["HOURLYRelativeHumidity"])

{nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 4.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 96.0,
 97.0,
 100.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 5.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,

In [86]:
wrapper = lambda row: heatIndex(row["HOURLYDRYBULBTEMPF"], row["HOURLYRelativeHumidity"])
hourlySummaries["HOURLYHeatIndex"] = hourlySummaries.apply(wrapper, axis=1)

# export hourly data
hourlySummaries.to_csv(path_or_buf="HourlyWeather.csv")

Examining the Daily weather dataset.

In [87]:
dailySummaries.head()

,DATE,DAILYMaximumDryBulbTemp,DAILYMinimumDryBulbTemp,DAILYAverageDryBulbTemp,DAILYAverageRelativeHumidity,DAILYSunrise,DAILYSunset,DAILYPrecip,DAILYSnowfall,DAILYSnowDepth,DAILYAverageWindSpeed
24,2010-01-01 23:59,70.0,51.0,60.0,44.0,659,1656,0.00,NaN,NaN,0.9
49,2010-01-02 23:59,78.0,50.0,64.0,35.0,659,1657,0.00,NaN,NaN,0.9
74,2010-01-03 23:59,79.0,54.0,66.0,24.0,659,1657,0.00,NaN,NaN,1.3
99,2010-01-04 23:59,77.0,56.0,66.0,24.0,659,1658,0.00,NaN,NaN,2.3
124,2010-01-05 23:59,78.0,54.0,66.0,34.0,659,1659,0.00,NaN,NaN,1.6


In [88]:
dailySummaries["DAILYAverageDryBulbTemp"] = dailySummaries["DAILYAverageDryBulbTemp"].apply(conv)
dailySummaries["DAILYMaximumDryBulbTemp"] = dailySummaries["DAILYMaximumDryBulbTemp"].apply(conv)
dailySummaries["DAILYMinimumDryBulbTemp"] = dailySummaries["DAILYMinimumDryBulbTemp"].apply(conv)
dailySummaries["DAILYAverageRelativeHumidity"] = dailySummaries["DAILYAverageRelativeHumidity"].apply(conv)
dailySummaries["DAILYPrecip"] = dailySummaries["DAILYPrecip"].apply(conv)

In [96]:
wrapper2 = lambda row: heatIndex(row["DAILYAverageDryBulbTemp"], row["DAILYAverageRelativeHumidity"])
dailySummaries["DAILYAverageHeatIndex"] = dailySummaries.apply(wrapper2, axis=1)

In [97]:
# Export
dailySummaries.to_csv(path_or_buf="DailyWeather.csv")